In [2]:
%matplotlib inline
import scipy.io as scio
import matplotlib.pyplot as plt
import matplotlib as mpl
from scipy.signal import savgol_filter
import cartopy.feature as cfeature
import cartopy.crs as ccrs
import netCDF4 as nc
import numpy as np
from Load_data import Data_from_nc
import xarray as xr
import math
import gc
from scipy.stats import linregress
from scipy import stats
from typing import Sequence, Tuple
from typing import Dict, Tuple
# ─── I/O Helpers ────────────────────────────────────────────────────────────

def get_data_from_mat_for_calcu(file, variable):
    var_dict = scio.loadmat(file)
    var = var_dict[variable]
    var = var[:, 29:]
    return var.T

def get_data_from_nc(file, variable):
    file_obj = nc.Dataset(file)
    data = file_obj.variables[variable]
    var_data = np.array(data)
    var_data = var_data[:, 29:, :]
    var_data[var_data > 1000000] = np.nan
    var_data = np.squeeze(var_data)
    return var_data

def get_data_from_mat(file, variable):
    var_dict = scio.loadmat(file)
    var = var_dict[variable]
    var = var[:, 29:] 
    return var

str_base = '' #TODO 
ar6_region = get_data_from_mat_for_calcu(str_base + '/plotting_tools/ar6_region.mat','ar6_region')
AREA = get_data_from_mat(str_base + '/plotting_tools/AREA.mat', 'AREA') # Area for CLM_data
irr_diff = get_data_from_mat_for_calcu(str_base + '/plotting_tools/irr_diff_out.mat', 'irr_diff_out')

# ─── Model‐specific loader ──────────────────────────────────────────────────
# Here we have a function for each ESM to read the variable
# This is because the folders and the names are all model-specific
def get_data_cesm2(variable):
    
    str_start = '/water_fluxes/CESM2/CESM2_'
    str_mid = '_1901_2014_'
    str_end = '_yearmean'
    
    data_irr01 = get_data_from_nc(str_base + str_start + 'IRR01' + str_mid + variable + str_end, variable)
    data_noi01 = get_data_from_nc(str_base + str_start + 'NOI01' + str_mid + variable + str_end, variable)

    data_irr02 = get_data_from_nc(str_base + str_start + 'IRR02' + str_mid + variable + str_end, variable)
    data_noi02 = get_data_from_nc(str_base + str_start + 'NOI02' + str_mid + variable + str_end, variable)

    data_irr03 = get_data_from_nc(str_base + str_start + 'IRR03' + str_mid + variable + str_end, variable)
    data_noi03 = get_data_from_nc(str_base + str_start + 'NOI03' + str_mid + variable + str_end, variable)

    data_irr = (data_irr01 + data_irr02 + data_irr03) / 3 * 365 * 86400
    data_noi = (data_noi01 + data_noi02 + data_noi03) / 3 * 365 * 86400
    
    return data_irr[:-1,:,:], data_noi[:-1,:,:]

def get_data_e3sm(variable3):
    
    str_start = '/water_fluxes/E3SMv2/E3SM_'
    str_mid = '_1901_2014_'
    str_end = '_yearmean_0.9x1.25'
    
    data_irr01 = get_data_from_nc(str_base + 'IRR01' + str_mid + variable3 + str_end, variable3)
    data_noi01 = get_data_from_nc(str_base + 'NOI01' + str_mid + variable3 + str_end, variable3)

    data_irr02 = get_data_from_nc(str_base + 'IRR02' + str_mid + variable3 + str_end, variable3)
    data_noi02 = get_data_from_nc(str_base + 'NOI02' + str_mid + variable3 + str_end, variable3)

    data_irr = (data_irr01 + data_irr02) / 2 * 365 * 86400
    data_noi = (data_noi01 + data_noi02) / 2 * 365 * 86400

    return data_irr[6:-1,:,:], data_noi[6:-1,:,:]

def get_data_cesm2_gw(variable):
    
    str_start = '/water_fluxes/CESM2_gw/CESM2_gw_'
    str_mid = '_1901_2014_'
    str_end = '_yearmean'
    
    data_irr01 = get_data_from_nc(str_base + str_start + 'IRR01' + str_mid + variable + str_end, variable)
    data_noi01 = get_data_from_nc(str_base + str_start + 'NOI01' + str_mid + variable + str_end, variable)

    data_irr02 = get_data_from_nc(str_base + str_start + 'IRR02' + str_mid + variable + str_end, variable)
    data_noi02 = get_data_from_nc(str_base + str_start + 'NOI02' + str_mid + variable + str_end, variable)

    data_irr03 = get_data_from_nc(str_base + str_start + 'IRR03' + str_mid + variable + str_end, variable)
    data_noi03 = get_data_from_nc(str_base + str_start + 'NOI03' + str_mid + variable + str_end, variable)

    data_irr = (data_irr01 + data_irr02 + data_irr03) / 3 * 365 * 86400
    data_noi = (data_noi01 + data_noi02 + data_noi03) / 3 * 365 * 86400
    
    return data_irr[:-1,:,:], data_noi[:-1,:,:]
    
def get_data_noresm(variable):
    
    str_start = '/water_fluxes/NorESM2/NorESM_'
    str_mid = '_1901_2014_'
    str_end = '_yearmean'
    
    data_irr01 = get_data_from_nc(str_base + str_start + 'IRR01' + str_mid + variable + str_end, variable)
    data_noi01 = get_data_from_nc(str_base + str_start + 'NOI01' + str_mid + variable + str_end, variable)

    data_irr02 = get_data_from_nc(str_base + str_start + 'IRR02' + str_mid + variable + str_end, variable)
    data_noi02 = get_data_from_nc(str_base + str_start + 'NOI02' + str_mid + variable + str_end, variable)

    data_irr03 = get_data_from_nc(str_base + str_start + 'IRR03' + str_mid + variable + str_end, variable)
    data_noi03 = get_data_from_nc(str_base + str_start + 'NOI03' + str_mid + variable + str_end, variable)

    data_irr01_temp = np.zeros([116,163,288])# the first year of ensemble 01 is missing
    data_irr01_temp[1:, :, :] = data_irr01
    data_irr01_temp[0, :, :] = (data_irr02[0, :, :] + data_irr03[0, :, :])/2
    data_irr01 = data_irr01_temp
    
    data_irr = (data_irr01 + data_irr02 + data_irr03) / 3 * 365 * 86400
    data_noi = (data_noi01 + data_noi02 + data_noi03) / 3 * 365 * 86400
    
    return data_irr[1:-1,:,:], data_noi[1:-1,:,:]
    
    
    
def get_data_ipsl(variable2):
    
    str_start = '/water_fluxes/IPSL-CM6/'
    str_end = '_1901_2014_Month.nc_yearmean_0.9x1.25'
    
    data_irr = get_data_from_nc(str_base + str_start + 'IRR01_'+ variable2 + str_end, variable2)
    data_noi = get_data_from_nc(str_base + str_start + 'NOI01_'+ variable2 + str_end, variable2)


    data_irr = data_irr * 365 * 86400
    data_noi = data_noi * 365 * 86400

    return data_irr, data_noi

def get_data_cnrm(variable2):
    
    str_start = '/water_fluxes/CNRM-CM6-1/'
    
    str_end = '.nc_yearmean_yearmean_0.9x1.25'
    
    data_irr01 = get_data_from_nc(str_start + variable2 + '_IRR01' + str_end, variable2)
    data_noi01 = get_data_from_nc(str_start + variable2 + '_NOI01' + str_end, variable2)

    data_irr02 = get_data_from_nc(str_start + variable2 + '_IRR02' + str_end, variable2)
    data_noi02 = get_data_from_nc(str_start + variable2 + '_NOI02' + str_end, variable2)

    data_irr03 = get_data_from_nc(str_start + variable2 + '_IRR03' + str_end, variable2)
    data_noi03 = get_data_from_nc(str_start + variable2 + '_NOI03' + str_end, variable2)

    data_irr04 = get_data_from_nc(str_start + variable2 + '_IRR04' + str_end, variable2)
    data_noi04 = get_data_from_nc(str_start + variable2 + '_NOI04' + str_end, variable2)

    data_irr05 = get_data_from_nc(str_start + variable2 + '_IRR05' + str_end, variable2)
    data_noi05 = get_data_from_nc(str_start + variable2 + '_NOI05' + str_end, variable2)

 
    
    data_irr = (data_irr01+data_irr02+data_irr03+data_irr04+data_irr05) / 5 * 365 * 86400
    data_noi = (data_noi01+data_noi02+data_noi03+data_noi04+data_noi05) / 5 * 365 * 86400

    return data_irr, data_noi
    
    
def get_data_miroc(variable):
    
    str_start = '/water_fluxes/MIROC-INTEG-ES/'
    
    str_mid = '_mon_MIROC_'
    str_end = '_1901-2014.nc_0.9x1.25_yearmean'
    
    data_irr01 = get_data_from_nc(str_start + 'tranirr-01' + variable + str_mid + 'IRR01' + str_end, variable)
    data_noi01 = get_data_from_nc(str_start + '1901irr-01' + variable + str_mid + 'NOI01' + str_end, variable)

    data_irr02 = get_data_from_nc(str_start + 'tranirr-02' + variable + str_mid + 'IRR02' + str_end, variable)
    data_noi02 = get_data_from_nc(str_start + '1901irr-02' + variable + str_mid + 'NOI02' + str_end, variable)

    data_irr03 = get_data_from_nc(str_start + 'tranirr-03' + variable + str_mid + 'IRR03' + str_end, variable)
    data_noi03 = get_data_from_nc(str_start + '1901irr-03' + variable + str_mid + 'NOI03' + str_end, variable)

    data_irr = (data_irr01 + data_irr02 + data_irr03) / 3 * 365 * 86400
    data_noi = (data_noi01 + data_noi02 + data_noi03) / 3 * 365 * 86400
    
    return data_irr, data_noi

In [3]:
MODEL_LOADERS = {
    'CESM2':    get_data_cesm2,
    'CESM2_gw': get_data_cesm2_gw,
    'NorESM':   get_data_noresm,
    'E3SM':     get_data_e3sm,
    'IPSL':     get_data_ipsl,
    'CNRM':     get_data_cnrm,
    'MIROC':    get_data_miroc,
}


# specify per-model which variable names to fetch
# tuple of (rain_var, snow_var) means load both and sum
# single-member tuple means loader returns P directly
PREC_VARS = {
    'CESM2':    ('RAIN_FROM_ATM',  'SNOW_FROM_ATM'),
    'CESM2_gw': ('RAIN_FROM_ATM',  'SNOW_FROM_ATM'),
    'NorESM':   ('RAIN_FROM_ATM',  'SNOW_FROM_ATM'),
    'E3SM':     ('RAIN',           'SNOW'),
    'IPSL':     ('pr',),    # loader returns P already
    'CNRM':     ('pr',),
    'MIROC':    ('pr',),
}

# container for all precipitation datasets
precip = {}

for model, vars_ in PREC_VARS.items():
    loader = MODEL_LOADERS[model]
    # if we have separate rain & snow variables, fetch both & sum
    if len(vars_) == 2:
        rain_var, snow_var = vars_
        irr_r = loader(rain_var)[0]  # [0] = IRR
        noi_r = loader(rain_var)[1]  # [1] = NOI
        irr_s = loader(snow_var)[0]
        noi_s = loader(snow_var)[1]

        precip[model] = {
            'IRR': irr_r + irr_s,
            'NOI': noi_r + noi_s
        }
    else:
        # single var: loader returns precipitation directly
        (var,) = vars_
        irr_p, noi_p = loader(var)
        precip[model] = {
            'IRR': irr_p,
            'NOI': noi_p
        }

# Example access:
# IWW_IRR_1901_2014_CESM2_P   = precip['CESM2']['IRR']
# IWW_NOI_1901_2014_CESM2_P   = precip['CESM2']['NOI']
# IWW_IRR_1901_2014_E3SM_P    = precip['E3SM']['IRR']
# IWW_NOI_1901_2014_MIROC_P   = precip['MIROC']['NOI']

RUNOFF_VARS = {
    'CESM2':    'QRUNOFF',
    'CESM2_gw': 'QRUNOFF',
    'NorESM':   'QRUNOFF',
    'E3SM':     'QRUNOFF',
    'IPSL':     'mrro',
    'CNRM':     'mrro',
    'MIROC':    'mrro',
}

# container for all runoff datasets
runoff = {}

for model, var in RUNOFF_VARS.items():
    loader = MODEL_LOADERS[model]
    irr_r, noi_r = loader(var)
    runoff[model] = {
        'IRR': irr_r,
        'NOI': noi_r
    }
    

ET_COMPONENTS = {
    'CESM2':    ['QFLX_EVAP_TOT'],
    'CESM2_gw': ['QFLX_EVAP_TOT'],
    'NorESM':   ['QFLX_EVAP_TOT'],
    'E3SM':     ['QSOIL', 'QVEGE', 'QVEGT'],
    'IPSL':     ['evspsbl'],
    'CNRM':     ['evspsbl'],
    'MIROC':    ['evspsbl', 'tran'],
}

evapotran = {}

for model, comps in ET_COMPONENTS.items():
    loader = MODEL_LOADERS[model]
    irr_parts = []
    noi_parts = []

    # load each component and accumulate
    for var in comps:
        irr_var, noi_var = loader(var)
        irr_parts.append(irr_var)
        noi_parts.append(noi_var)

    # sum all parts to get total ET
    total_irr = sum(irr_parts)
    total_noi = sum(noi_parts)

    evapotran[model] = {
        'IRR': total_irr,
        'NOI': total_noi
    }
    
DATA_GROUPS = {
    'P'  : precip,
    'R'  : runoff,
    'ET' : evapotran,
}

# Here we use CESM2 output for masking in case some models also output grid cells over the ocean

for name, group in DATA_GROUPS.items():
    # make mask from CESM2 IRR
    ref_mask = np.isnan(group['CESM2']['IRR'])
    # apply to every other model & both scenarios
    for model, scen_dict in group.items():
        if model == 'CESM2':
            continue
        for scen in ('IRR','NOI'):
            scen_dict[scen][ref_mask] = np.nan

In [4]:
def calcu_global_water(data): # This is the function to calculate global mean water fluxes (not used here in this script)
    
    str_area = str_base + '/plotting_tools/AREA.mat'
    
    area = get_data_from_mat(str_area, 'AREA')
    
    area_for_calcu = area.T
    
    area_for_calcu[np.isnan(data[0, :, :])] = np.nan
    
    
    data_globe = data * AREA.T
    
    data_globe = np.nansum(data_globe, axis=(1, 2))
    
    area_for_calcu = np.nansum(area_for_calcu, axis=(0, 1))
    
    return data_globe/area_for_calcu

def calcu_water_region(data, region_id): 
    
    # This is the function to calculate regional mean water fluxes (used here)
    
    str_area = str_base + '/plotting_tools/AREA.mat'
    
    area = get_data_from_mat(str_area, 'AREA') 
    
    # Area for CLM_data, we read it every time when calling this function
    # Because in Jupyter notebook the array will be changed even though only being used in another function
    
    area_for_calcu = area.T
    
    # build a mask of all points to set to NaN
    mask = (
        np.isnan(data[0, :, :])                                    # missing data
        | (np.abs(ar6_region - region_id) > 0.2)                   # outside [region_id–tol, region_id+tol]
    )
    
    area_for_calcu[mask] = np.nan

    # Keep only the area in the region
    
    data_region = data * area_for_calcu
    
    data_region[:, mask] = np.nan

    data_region = np.nansum(data_region, axis=(1, 2))

    area_for_calcu = np.nansum(area_for_calcu, axis=(0, 1))
    
    return data_region/area_for_calcu

In [5]:
# We divide the whole period (114) years to six 19-year periods
def get_19_years_mean(data):
    data_1=np.mean(data[0:19])
    data_2=np.mean(data[19:38])
    data_3=np.mean(data[38:57])
    data_4=np.mean(data[57:76])
    data_5=np.mean(data[76:95])
    data_6=np.mean(data[95:114])
    return np.array([data_1,data_2,data_3,data_4,data_5,data_6])

In [6]:
def get_regional_p(region_id):

    
    # 1. Define the models in the exact order you want them stacked
    models = [
        'CESM2',
        'CESM2_gw',
        'NorESM',
        'E3SM',
        'IPSL',
        'MIROC',
        'CNRM',
    ]

    # 2. Prepare empty lists to collect each model's IRR/NOI series
    irr_list = []
    noi_list = []

    # 3. Loop once, calculate both series (using your helper), and append
    for m in models:
        irr = calcu_water_region(
            precip[m]['IRR'],
            region_id
        )
        noi = calcu_water_region(
            precip[m]['NOI'],
            region_id
        )
        
        irr_list.append(irr)
        noi_list.append(noi)

    # 4. Stack them all in one go – same as your vstack calls
    data_irr_p_et = np.vstack(irr_list)
    data_noi_p_et = np.vstack(noi_list)

    # 5. Compute the difference
    data_diff_p_et = data_irr_p_et - data_noi_p_et
    
    return data_irr_p_et, data_noi_p_et, data_diff_p_et

def get_regional_et(region_id):

    
    # 1. Define the models in the exact order you want them stacked
    models = [
        'CESM2',
        'CESM2_gw',
        'NorESM',
        'E3SM',
        'IPSL',
        'MIROC',
        'CNRM',
    ]

    # 2. Prepare empty lists to collect each model's IRR/NOI series
    irr_list = []
    noi_list = []

    # 3. Loop once, calculate both series (using your helper), and append
    for m in models:
        irr = calcu_water_region(
            evapotran[m]['IRR'],
            region_id
        )
        noi = calcu_water_region(
            evapotran[m]['NOI'],
            region_id
        )
        
        irr_list.append(irr)
        noi_list.append(noi)

    # 4. Stack them all in one go – same as your vstack calls
    data_irr_p_et = np.vstack(irr_list)
    data_noi_p_et = np.vstack(noi_list)

    # 5. Compute the difference
    data_diff_p_et = data_irr_p_et - data_noi_p_et
    
    return data_irr_p_et, data_noi_p_et, data_diff_p_et

def get_regional_r(region_id):

    
    # 1. Define the models in the exact order you want them stacked
    models = [
        'CESM2',
        'CESM2_gw',
        'NorESM',
        'E3SM',
        'IPSL',
        'MIROC',
        'CNRM',
    ]

    # 2. Prepare empty lists to collect each model's IRR/NOI series
    irr_list = []
    noi_list = []

    # 3. Loop once, calculate both series (using your helper), and append
    for m in models:
        irr = calcu_water_region(
            runoff[m]['IRR'],
            region_id
        )
        noi = calcu_water_region(
            runoff[m]['NOI'],
            region_id
        )
        
        irr_list.append(irr)
        noi_list.append(noi)

    # 4. Stack them all in one go – same as your vstack calls
    data_irr_p_et = np.vstack(irr_list)
    data_noi_p_et = np.vstack(noi_list)

    # 5. Compute the difference
    data_diff_p_et = data_irr_p_et - data_noi_p_et
    
    return data_irr_p_et, data_noi_p_et, data_diff_p_et

In [7]:
# 1) map your region names → AR6 IDs
REGIONS = {
    'SAS': 38,
    'MED': 20,
    'CNA':  5,
    'WCA': 33,
}

def compute_stats(arr: np.ndarray, axis=0) -> dict[str, np.ndarray]:
    """
    Return the 25th, 50th, 75th percentiles and mean of `arr` along `axis`.
    """
    pcts = np.percentile(arr, [25, 50, 75], axis=axis)
    return {
        'p25': get_19_years_mean(pcts[0]),
        'p50': get_19_years_mean(pcts[1]),
        'p75': get_19_years_mean(pcts[2]),
        'mean': get_19_years_mean(np.mean(arr, axis=axis)),
    }

def batch_regional_stats_p(regions: dict[str,int]):
    """
    For each region in `regions`, call get_regional_P_E and compute stats
    for IRR, NOI, and their difference.
    Returns a nested dict: stats[region]['irr']['p75'], etc.
    """
    stats: dict[str, dict[str, dict[str, np.ndarray]]] = {}

    for name, rid in regions.items():
        irr_arr, noi_arr, diff_arr = get_regional_p(rid)

        stats[name] = {
            'irr':  compute_stats(irr_arr),
            'noi':  compute_stats(noi_arr),
            'diff': compute_stats(diff_arr),
        }

    return stats

def batch_regional_stats_et(regions: dict[str,int]):
    """
    For each region in `regions`, call get_regional_P_E and compute stats
    for IRR, NOI, and their difference.
    Returns a nested dict: stats[region]['irr']['p75'], etc.
    """
    stats: dict[str, dict[str, dict[str, np.ndarray]]] = {}

    for name, rid in regions.items():
        irr_arr, noi_arr, diff_arr = get_regional_et(rid)

        stats[name] = {
            'irr':  compute_stats(irr_arr),
            'noi':  compute_stats(noi_arr),
            'diff': compute_stats(diff_arr),
        }

    return stats

def batch_regional_stats_r(regions: dict[str,int]):
    """
    For each region in `regions`, call get_regional_P_E and compute stats
    for IRR, NOI, and their difference.
    Returns a nested dict: stats[region]['irr']['p75'], etc.
    """
    stats: dict[str, dict[str, dict[str, np.ndarray]]] = {}

    for name, rid in regions.items():
        irr_arr, noi_arr, diff_arr = get_regional_r(rid)

        stats[name] = {
            'irr':  compute_stats(irr_arr),
            'noi':  compute_stats(noi_arr),
            'diff': compute_stats(diff_arr),
        }

    return stats

# run it once
region_stats_p = batch_regional_stats_p(REGIONS)
region_stats_et = batch_regional_stats_et(REGIONS)
region_stats_r = batch_regional_stats_r(REGIONS)

In [ ]:
fig = plt.figure(figsize=(12, 12),dpi=300)
fig.subplots_adjust(hspace=0.4, wspace=0.4, left = 0.05, right = 0.95, top = 0.95, bottom = 0.05)
ax1 = plt.subplot(431, frameon=True)
# ax1.spines['top'].set_visible(False)
# ax1.spines['right'].set_visible(False)
X = np.array([1910, 1929, 1948, 1967, 1986, 2005])
plt.bar(X-3, region_stats_p['SAS']['irr']['mean'], yerr=[region_stats_p['SAS']['irr']['mean']-region_stats_p['SAS']['irr']['p25'], region_stats_p['SAS']['irr']['p75']-region_stats_p['SAS']['irr']['mean']], width = 6, color='dodgerblue', capsize=6,label='P tranirr',alpha=0.8)
plt.bar(X+3, region_stats_p['SAS']['noi']['mean'], yerr=[region_stats_p['SAS']['noi']['mean']-region_stats_p['SAS']['noi']['p25'], region_stats_p['SAS']['noi']['p75']-region_stats_p['SAS']['noi']['mean']], width = 6, color='aqua', capsize=6,label='P 1901irr',alpha=0.8)
plt.xticks(X, ['I','II','III','IV','V','VI'], fontsize=10)
plt.yticks(fontsize=10)
plt.legend(fontsize=10, loc='lower right')
plt.xlim(1901, 2015)
plt.ylabel('P (mm/year)', fontsize=12)
plt.title('South Asia', fontsize=14, loc='center')
plt.title('a', fontsize=14, loc='left')

ax1 = plt.subplot(432, frameon=True)
plt.bar(X-3, region_stats_et['SAS']['irr']['mean'], yerr=[region_stats_et['SAS']['irr']['mean']-region_stats_et['SAS']['irr']['p25'], region_stats_et['SAS']['irr']['p75']-region_stats_et['SAS']['irr']['mean']], width = 6, color='tomato', capsize=6,label='ET tranirr',alpha=0.8)
plt.bar(X+3, region_stats_et['SAS']['noi']['mean'], yerr=[region_stats_et['SAS']['noi']['mean']-region_stats_et['SAS']['noi']['p25'], region_stats_et['SAS']['noi']['p75']-region_stats_et['SAS']['noi']['mean']], width = 6, color='darksalmon', capsize=6,label='ET 1901irr',alpha=0.8)
plt.xticks(X, ['I','II','III','IV','V','VI'], fontsize=10)
plt.yticks(fontsize=10)
plt.legend(fontsize=10, loc='lower right')
plt.xlim(1901, 2015)
plt.ylabel('ET (mm/year)', fontsize=12)
plt.title('South Asia', fontsize=14, loc='center')
plt.title('b', fontsize=14, loc='left')

ax1 = plt.subplot(433, frameon=True)
plt.bar(X-3, region_stats_r['SAS']['irr']['mean'], yerr=[region_stats_r['SAS']['irr']['mean']-region_stats_r['SAS']['irr']['p25'], region_stats_r['SAS']['irr']['p75']-region_stats_r['SAS']['irr']['mean']], width = 6, color='lime', capsize=6,label='R tranirr',alpha=0.8)
plt.bar(X+3, region_stats_r['SAS']['noi']['mean'], yerr=[region_stats_r['SAS']['noi']['mean']-region_stats_r['SAS']['noi']['p25'], region_stats_r['SAS']['noi']['p75']-region_stats_r['SAS']['noi']['mean']], width = 6, color='lightgreen', capsize=6,label='R 1901irr',alpha=0.8)
plt.xticks(X, ['I','II','III','IV','V','VI'], fontsize=10)
plt.yticks(fontsize=10)
plt.legend(fontsize=10, loc='lower right')
plt.xlim(1901, 2015)
plt.ylabel('R (mm/year)', fontsize=12)
plt.title('South Asia', fontsize=14, loc='center')
plt.title('c', fontsize=14, loc='left')


ax1 = plt.subplot(434, frameon=True)
plt.bar(X-3, region_stats_p['MED']['irr']['mean'], yerr=[region_stats_p['MED']['irr']['mean']-region_stats_p['MED']['irr']['p25'], region_stats_p['MED']['irr']['p75']-region_stats_p['MED']['irr']['mean']], width = 6, color='dodgerblue', capsize=6,label='P tranirr',alpha=0.8)
plt.bar(X+3, region_stats_p['MED']['noi']['mean'], yerr=[region_stats_p['MED']['noi']['mean']-region_stats_p['MED']['noi']['p25'], region_stats_p['MED']['noi']['p75']-region_stats_p['MED']['noi']['mean']], width = 6, color='aqua', capsize=6,label='P 1901irr',alpha=0.8)
plt.xticks(X, ['I','II','III','IV','V','VI'], fontsize=10)
plt.yticks(fontsize=10)
# plt.legend(fontsize=10, loc='lower right')
plt.xlim(1901, 2015)
plt.ylabel('P (mm/year)', fontsize=12)
plt.title('Mediterranean', fontsize=14, loc='center')
plt.title('d', fontsize=14, loc='left')

ax1 = plt.subplot(435, frameon=True)
plt.bar(X-3, region_stats_et['MED']['irr']['mean'], yerr=[region_stats_et['MED']['irr']['mean']-region_stats_et['MED']['irr']['p25'], region_stats_et['MED']['irr']['p75']-region_stats_et['MED']['irr']['mean']], width = 6, color='tomato', capsize=6,label='ET tranirr',alpha=0.8)
plt.bar(X+3, region_stats_et['MED']['noi']['mean'], yerr=[region_stats_et['MED']['noi']['mean']-region_stats_et['MED']['noi']['p25'], region_stats_et['MED']['noi']['p75']-region_stats_et['MED']['noi']['mean']], width = 6, color='darksalmon', capsize=6,label='ET 1901irr',alpha=0.8)
plt.xticks(X, ['I','II','III','IV','V','VI'], fontsize=10)
plt.yticks(fontsize=10)
# plt.legend(fontsize=10, loc='lower right')
plt.xlim(1901, 2015)
plt.ylabel('ET (mm/year)', fontsize=12)
plt.title('Mediterranean', fontsize=14, loc='center')
plt.title('e', fontsize=14, loc='left')

ax1 = plt.subplot(436, frameon=True)
plt.bar(X-3, region_stats_r['MED']['irr']['mean'], yerr=[region_stats_r['MED']['irr']['mean']-region_stats_r['MED']['irr']['p25'], region_stats_r['MED']['irr']['p75']-region_stats_r['MED']['irr']['mean']], width = 6, color='lime', capsize=6,label='R tranirr',alpha=0.8)
plt.bar(X+3, region_stats_r['MED']['noi']['mean'], yerr=[region_stats_r['MED']['noi']['mean']-region_stats_r['MED']['noi']['p25'], region_stats_r['MED']['noi']['p75']-region_stats_r['MED']['noi']['mean']], width = 6, color='lightgreen', capsize=6,label='R 1901irr',alpha=0.8)
plt.xticks(X, ['I','II','III','IV','V','VI'], fontsize=10)
plt.yticks(fontsize=10)
# plt.legend(fontsize=10, loc='lower right')
plt.xlim(1901, 2015)
plt.ylabel('R (mm/year)', fontsize=12)
plt.title('Mediterranean', fontsize=14, loc='center')
plt.title('f', fontsize=14, loc='left')

ax1 = plt.subplot(437, frameon=True)
plt.bar(X-3, region_stats_p['CNA']['irr']['mean'], yerr=[region_stats_p['CNA']['irr']['mean']-region_stats_p['CNA']['irr']['p25'], region_stats_p['CNA']['irr']['p75']-region_stats_p['CNA']['irr']['mean']], width = 6, color='dodgerblue', capsize=6,label='P tranirr',alpha=0.8)
plt.bar(X+3, region_stats_p['CNA']['noi']['mean'], yerr=[region_stats_p['CNA']['noi']['mean']-region_stats_p['CNA']['noi']['p25'], region_stats_p['CNA']['noi']['p75']-region_stats_p['CNA']['noi']['mean']], width = 6, color='aqua', capsize=6,label='P 1901irr',alpha=0.8)
plt.xticks(X, ['I','II','III','IV','V','VI'], fontsize=10)
plt.yticks(fontsize=10)
# plt.legend(fontsize=10, loc='lower right')
plt.xlim(1901, 2015)
plt.ylabel('P (mm/year)', fontsize=12)
plt.title('Central North America', fontsize=14, loc='center')
plt.title('g', fontsize=14, loc='left')

ax1 = plt.subplot(438, frameon=True)
plt.bar(X-3, region_stats_et['CNA']['irr']['mean'], yerr=[region_stats_et['CNA']['irr']['mean']-region_stats_et['CNA']['irr']['p25'], region_stats_et['CNA']['irr']['p75']-region_stats_et['CNA']['irr']['mean']], width = 6, color='tomato', capsize=6,label='ET tranirr',alpha=0.8)
plt.bar(X+3, region_stats_et['CNA']['noi']['mean'], yerr=[region_stats_et['CNA']['noi']['mean']-region_stats_et['CNA']['noi']['p25'], region_stats_et['CNA']['noi']['p75']-region_stats_et['CNA']['noi']['mean']], width = 6, color='darksalmon', capsize=6,label='ET 1901irr',alpha=0.8)
plt.xticks(X, ['I','II','III','IV','V','VI'], fontsize=10)
plt.yticks(fontsize=10)
# plt.legend(fontsize=10, loc='lower right')
plt.xlim(1901, 2015)
plt.ylabel('ET (mm/year)', fontsize=12)
plt.title('Central North America', fontsize=14, loc='center')
plt.title('h', fontsize=14, loc='left')

ax1 = plt.subplot(439, frameon=True)
plt.bar(X-3, region_stats_r['CNA']['irr']['mean'], yerr=[region_stats_r['CNA']['irr']['mean']-region_stats_r['CNA']['irr']['p25'], region_stats_r['CNA']['irr']['p75']-region_stats_r['CNA']['irr']['mean']], width = 6, color='lime', capsize=6,label='R tranirr',alpha=0.8)
plt.bar(X+3, region_stats_r['CNA']['noi']['mean'], yerr=[region_stats_r['CNA']['noi']['mean']-region_stats_r['CNA']['noi']['p25'], region_stats_r['CNA']['noi']['p75']-region_stats_r['CNA']['noi']['mean']], width = 6, color='lightgreen', capsize=6,label='R 1901irr',alpha=0.8)
plt.xticks(X, ['I','II','III','IV','V','VI'], fontsize=10)
plt.yticks(fontsize=10)
# plt.legend(fontsize=10, loc='lower right')
plt.xlim(1901, 2015)
plt.ylabel('R (mm/year)', fontsize=12)
plt.title('Central North America', fontsize=14, loc='center')
plt.title('i', fontsize=14, loc='left')

ax1 = plt.subplot(4,3,10, frameon=True)
plt.bar(X-3, region_stats_p['WCA']['irr']['mean'], yerr=[region_stats_p['WCA']['irr']['mean']-region_stats_p['WCA']['irr']['p25'], region_stats_p['WCA']['irr']['p75']-region_stats_p['WCA']['irr']['mean']], width = 6, color='dodgerblue', capsize=6,label='P tranirr',alpha=0.8)
plt.bar(X+3, region_stats_p['WCA']['noi']['mean'], yerr=[region_stats_p['WCA']['noi']['mean']-region_stats_p['WCA']['noi']['p25'], region_stats_p['WCA']['noi']['p75']-region_stats_p['WCA']['noi']['mean']], width = 6, color='aqua', capsize=6,label='P 1901irr',alpha=0.8)
plt.xticks(X, ['I','II','III','IV','V','VI'], fontsize=10)
plt.yticks(fontsize=10)
# plt.legend(fontsize=10, loc='lower right')
plt.xlim(1901, 2015)
plt.ylabel('P (mm/year)', fontsize=12)
plt.title('West Central Asia', fontsize=14, loc='center')
plt.title('j', fontsize=14, loc='left')

ax1 = plt.subplot(4,3,11, frameon=True)
plt.bar(X-3, region_stats_et['WCA']['irr']['mean'], yerr=[region_stats_et['WCA']['irr']['mean']-region_stats_et['WCA']['irr']['p25'], region_stats_et['WCA']['irr']['p75']-region_stats_et['WCA']['irr']['mean']], width = 6, color='tomato', capsize=6,label='ET tranirr',alpha=0.8)
plt.bar(X+3, region_stats_et['WCA']['noi']['mean'], yerr=[region_stats_et['WCA']['noi']['mean']-region_stats_et['WCA']['noi']['p25'], region_stats_et['WCA']['noi']['p75']-region_stats_et['WCA']['noi']['mean']], width = 6, color='darksalmon', capsize=6,label='ET 1901irr',alpha=0.8)
plt.xticks(X, ['I','II','III','IV','V','VI'], fontsize=10)
plt.yticks(fontsize=10)
# plt.legend(fontsize=10, loc='lower right')
plt.xlim(1901, 2015)
plt.ylabel('ET (mm/year)', fontsize=12)
plt.title('West Central Asia', fontsize=14, loc='center')
plt.title('k', fontsize=14, loc='left')

ax1 = plt.subplot(4,3,12, frameon=True)
plt.bar(X-3, region_stats_r['WCA']['irr']['mean'], yerr=[region_stats_r['WCA']['irr']['mean']-region_stats_r['WCA']['irr']['p25'], region_stats_r['WCA']['irr']['p75']-region_stats_r['WCA']['irr']['mean']], width = 6, color='lime', capsize=6,label='R tranirr',alpha=0.8)
plt.bar(X+3, region_stats_r['WCA']['noi']['mean'], yerr=[region_stats_r['WCA']['noi']['mean']-region_stats_r['WCA']['noi']['p25'], region_stats_r['WCA']['noi']['p75']-region_stats_r['WCA']['noi']['mean']], width = 6, color='lightgreen', capsize=6,label='R 1901irr',alpha=0.8)
plt.xticks(X, ['I','II','III','IV','V','VI'], fontsize=10)
plt.yticks(fontsize=10)
# plt.legend(fontsize=10, loc='lower right')
plt.xlim(1901, 2015)
plt.ylabel('R (mm/year)', fontsize=12)
plt.title('West Central Asia', fontsize=14, loc='center')
plt.title('l', fontsize=14, loc='left')